In [186]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

In [187]:
stroke_data = pd.read_csv("healthcare-dataset-stroke-data.csv")
stroke_data.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,4909.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,28.893237,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,7.854067,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,23.500000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,28.100000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,33.100000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


In [188]:
# Preprocessing.

df = stroke_data.copy()
df_data = df.drop(columns=["stroke", "id"])
df_target = df["stroke"]

# Let's first impute the missing values.
# We have two columns with missing values:
# The bmi column has missing values in the form of NaN values; we will replace these with the column's mean.
# The smoking_status column has missing values in the form of "Unknown" strings; we will replace these with the column's mode.

imp_bmi = SimpleImputer(strategy='mean')
imp_smoking_status = SimpleImputer(missing_values="Unknown", strategy='most_frequent')

df_data["bmi"] = imp_bmi.fit_transform(df_data[["bmi"]])
df_data["smoking_status"] = imp_smoking_status.fit_transform(df_data[["smoking_status"]])

# Now we will use one-hot encoding to convert categorical vars into numeric vars
encoded_features = pd.get_dummies(df_data[['gender', 'work_type', 'Residence_type', 'smoking_status', 'ever_married']], drop_first=True)

# Concatenate the encoded features with the original DataFrame
df_data = pd.concat([df_data, encoded_features], axis=1)

# Drop the original categorical columns
df_data.drop(['gender', 'work_type', 'Residence_type', 'smoking_status', 'ever_married'], axis=1, inplace=True)

X_raw = df_data.values
y_raw = df_target

# Splitting the test and training data
X_train_raw, X_test, y_train_raw, y_test = train_test_split(X_raw, y_raw, test_size=0.3, stratify=y, random_state=734981)


# Next, let's fix the class imbalance in our dataset by oversampling the minority class.
ros = RandomOverSampler(random_state=2873159)
X_train, y_train = ros.fit_resample(X_train_raw, y_train_raw)

# We can see that the classes are now balanced.
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{0: 3403, 1: 3403}

Random forest implementation & hyper-parameter tuning:

Hyper-parameter tuning using sklearn's RandomizedSearchCV

In [205]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier  

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 2000, num = 15)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 200, num = 20)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 200, cv = 3, verbose=0, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

In [190]:
print(rf_random.best_params_)

{'n_estimators': 1025, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 80, 'bootstrap': False}


In [204]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Create base (untuned)rf model
rf_base = RandomForestClassifier()  
rf_base.fit(X_train, y_train)

predictions_base = rf_base.predict(X_test)

# Compute accuracy & F1 score on base model
accuracy_base = accuracy_score(y_test, predictions_base)
f1_base = f1_score(y_test, predictions_base)
precision_base = precision_score(y_test, predictions_base)
recall_base = recall_score(y_test, predictions_base)
print(f"Accuracy (base): {round(accuracy_base, 4)}%")
print(f"Precision (base): {round(precision_base, 4)}%")
print(f"Recall (base): {round(recall_base, 4)}%")
print(f"F1 Score (base): {round(f1_base, 4)}")
print()

# Compute accuracy & F1 score on tuned model
rf_tuned = RandomForestClassifier(n_estimators=1025, min_samples_split=2, min_samples_leaf=5, max_features= 'sqrt', max_depth=80, bootstrap=False)  
rf_tuned.fit(X_train, y_train)
predictions_tuned = rf_tuned.predict(X_test)
accuracy_tuned = accuracy_score(y_test, predictions_tuned)
f1_tuned = f1_score(y_test, predictions_tuned)
precision_tuned = precision_score(y_test, predictions_tuned)
recall_tuned = recall_score(y_test, predictions_tuned)
print(f"Accuracy (new): {round(accuracy_tuned, 4)}%")
print(f"Precision (tuned): {round(precision_tuned, 4)}%")
print(f"Recall (new): {round(recall_tuned, 4)}%")
print(f"F1 Score (new): {round(f1_tuned, 4)}")

Accuracy (base): 0.9413%
Precision (base): 0.0588%
Recall (base): 0.0133%
F1 Score (base): 0.0217

Accuracy (new): 0.9106%
Precision (tuned): 0.122%
Recall (new): 0.1333%
F1 Score (new): 0.1274


The accuracy had slightly reduced in the new model, but the F1 score has improved, which is good because we want to maximise the recall and precision on the positive class. Notice that we tweaked <i>min_samples_leaf=5</i> manually so that the depth of each tree is restricted, which reduces overfitting and happens to signficantly increase the recall of our model. Recall is important because we prioritise correctly classifying examples from the positive (stroke) class (because false negatives are more dangerous than false positives in the given domain).

In [216]:
most_important_features = rf_tuned.feature_importances_

feature_importance_map = {}
for feature_index in range(0, len(most_important_features)):
    feature_name = df_data.columns[feature_index]
    feature_importance_map[feature_name] = most_important_features[feature_index]

features_sorted_by_importance = sorted(feature_importance_map.items(), key=lambda x: x[1], reverse=True)
print(features_sorted_by_importance)
top_three_important_features = features_sorted_by_importance[:3]
print(top_three_important_features)

[('age', 0.41057665332887977), ('avg_glucose_level', 0.19538128779604252), ('bmi', 0.17244253629827658), ('ever_married_Yes', 0.040372291983265), ('hypertension', 0.032161433084419204), ('heart_disease', 0.028873945436842255), ('gender_Male', 0.02104858890638547), ('work_type_children', 0.018769887353160205), ('work_type_Private', 0.018201969528491777), ('Residence_type_Urban', 0.017984730119045847), ('smoking_status_never smoked', 0.017911171216334613), ('work_type_Self-employed', 0.013253505797743382), ('smoking_status_smokes', 0.01283659741378613), ('work_type_Never_worked', 0.00018540173732720425), ('gender_Other', 0.0)]
[('age', 0.41057665332887977), ('avg_glucose_level', 0.19538128779604252), ('bmi', 0.17244253629827658)]
